***⚙️ Imports & Helpers***

In [9]:
import os
import re
import json
from datetime import datetime, timedelta
from collections import defaultdict
from pathlib import Path

import folium
import branca.colormap as cm
import geopandas as gpd
from shapely.geometry import LineString, Point
from tqdm import tqdm
from pyproj import Transformer
from pyswmm import Simulation, Nodes, Links

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.dates as mdates
from matplotlib import colors, colormaps

from ipywidgets import Dropdown, IntSlider, HBox, VBox, Output as ipyOutput, Output
from IPython.display import display, clear_output, HTML

def parse_inp_file(inp_path):
    """Parse SWMM .inp file to extract network information"""
    with open(inp_path) as f:
        content = f.read()

    data = {
        'coordinates': {},
        'vertices': defaultdict(list),
        'conduits': [],
        'junctions': set(),
        'storage': set(),
        'outfalls': set()
    }

    patterns = {
        'coordinates': r"\[COORDINATES\](.*?)(\n\[|\Z)",
        'vertices': r"\[VERTICES\](.*?)(\n\[|\Z)",
        'conduits': r"\[CONDUITS\](.*?)(\n\[|\Z)",
        'junctions': r"\[JUNCTIONS\](.*?)(\n\[|\Z)",
        'storage': r"\[STORAGE\](.*?)(\n\[|\Z)",
        'outfalls': r"\[OUTFALLS\](.*?)(\n\[|\Z)"
    }

    match = re.search(patterns['coordinates'], content, re.DOTALL)
    if match:
        for line in match.group(1).splitlines():
            if line.strip() and not line.strip().startswith(';'):
                nid, x, y = re.split(r'\s+', line.strip())[:3]
                data['coordinates'][nid] = [float(x), float(y)]

    match = re.search(patterns['vertices'], content, re.DOTALL)
    if match:
        for line in match.group(1).splitlines():
            if line.strip() and not line.strip().startswith(';'):
                cid, x, y = re.split(r'\s+', line.strip())[:3]
                data['vertices'][cid].append([float(x), float(y)])

    match = re.search(patterns['conduits'], content, re.DOTALL)
    if match:
        for line in match.group(1).splitlines():
            if line.strip() and not line.strip().startswith(';'):
                cid, from_node, to_node = re.split(r'\s+', line.strip())[:3]
                data['conduits'].append({
                    'id': cid, 'from_node': from_node, 'to_node': to_node
                })

    for key in ['junctions', 'storage', 'outfalls']:
        match = re.search(patterns[key], content, re.DOTALL)
        if match:
            for line in match.group(1).splitlines():
                if line.strip() and not line.strip().startswith(';'):
                    nid = re.split(r'\s+', line.strip())[0]
                    data[key].add(nid)

    return data

def format_datetime(dt):
    """Format datetime as YYYY/MM/DD HH:MM rounded to nearest 5 minutes"""
    rounded_minute = (dt.minute // 5) * 5
    rounded_dt = dt.replace(minute=rounded_minute, second=0, microsecond=0)
    return rounded_dt.strftime("%Y/%m/%d %H:%M")


***🧪 Run Simulation & Extract Data***

In [ ]:
def run_simulation_with_progress(inp_path, ini_path):
    """Run SWMM simulation with progress bar, collect data, and update .ini file"""
    print("Running simulation (this may take some time)...")
    
    # Initialize data structures
    sim_data = {
        'times': [],
        'node_data': defaultdict(lambda: defaultdict(list)),
        'link_data': defaultdict(lambda: defaultdict(list)),
        'conduit_timeseries': defaultdict(lambda: defaultdict(list))
    }

    # Run simulation with progress bar
    with Simulation(inp_path) as sim:
        nodes = Nodes(sim)
        links = Links(sim)
        
        node_map = {n.nodeid: n for n in nodes}
        link_map = {l.linkid: l for l in links}

        # Progress bar setup
        with tqdm(total=100, desc="Simulation Progress", ncols=80) as pbar:
            last_progress = 0
            for step in sim:
                # Update progress bar
                progress = int(sim.percent_complete * 100)
                if progress > last_progress:
                    pbar.update(progress - last_progress)
                    last_progress = progress

                # Collect simulation data
                current_time = sim.current_time
                sim_data['times'].append(current_time)
                
                # Node data
                for nid, node in node_map.items():
                    sim_data['node_data'][nid]['Depth (ft)'].append(node.depth)
                    sim_data['node_data'][nid]['Head (ft)'].append(node.head)
                    sim_data['node_data'][nid]['Total Inflow (cfs)'].append(node.total_inflow)
                    sim_data['node_data'][nid]['Lateral Inflow (cfs)'].append(node.lateral_inflow)
                    sim_data['node_data'][nid]['Surcharge Depth (ft)'].append(node.surcharge_depth)
                
                # Link data
                for lid, link in link_map.items():
                    sim_data['link_data'][lid]['Flow (cfs)'].append(link.flow)
                    sim_data['link_data'][lid]['Depth (ft)'].append(link.depth)
                    sim_data['link_data'][lid]['Volume (ft³)'].append(link.volume)
                    sim_data['conduit_timeseries']['FLOW'][lid].append(link.flow)
                    sim_data['conduit_timeseries']['DEPTH'][lid].append(link.depth)
                    sim_data['conduit_timeseries']['VOLUME'][lid].append(link.volume)

            # Ensure progress reaches 100%
            if last_progress < 100:
                pbar.update(100 - last_progress)

    # Update .ini file (set Current=1 under [Results])
    updated_lines = []
    inside_results = False
    with open(ini_path, 'r') as file:
        for line in file:
            if line.strip() == '[Results]':
                inside_results = True
                updated_lines.append(line)
            elif inside_results and line.strip().startswith('Current='):
                updated_lines.append('Current=1\n')
                inside_results = False
            else:
                updated_lines.append(line)

    with open(ini_path, 'w') as file:
        file.writelines(updated_lines)

    print("Simulation complete! Updated .ini file: 'Current=1' set under [Results]")
    return sim_data

# Execute
inp_path = "Example.inp" #SWMM input file
ini_path = "Example.ini" #SWMM configuration file
sim_data = run_simulation_with_progress(inp_path, ini_path)
network_data = parse_inp_file(inp_path)  # Ensure parse_inp_file() is defined

Running simulation (this may take some time)...


Simulation Progress: 100%|████████████████████| 100/100 [02:02<00:00,  1.23s/it]


Simulation complete! Updated .ini file: 'Current=1' set under [Results]


***🌐 GeoJSON Network File Extraction***

In [11]:
def export_geojson(coords, features, vertices, transformer, output_path, kind):
    geo_features = []

    if kind == "conduits":
        for feat in features:
            fid = feat['id']
            from_coords = coords.get(feat['from_node'])
            to_coords = coords.get(feat['to_node'])
            if not from_coords or not to_coords:
                continue

            line = []
            if fid in vertices:
                for v in vertices[fid]:
                    line.append(transformer.transform(*v) if transformer else v)

            line.insert(0, transformer.transform(*from_coords) if transformer else from_coords)
            line.append(transformer.transform(*to_coords) if transformer else to_coords)

            geo_features.append({
                "type": "Feature",
                "geometry": {"type": "LineString", "coordinates": line},
                "properties": {
                    "id": fid,
                    "from_node": feat['from_node'],
                    "to_node": feat['to_node'],
                    **feat['data']
                }
            })

    elif kind == "nodes":
        for nid, xy in coords.items():
            lon, lat = transformer.transform(*xy) if transformer else xy
            geo_features.append({
                "type": "Feature",
                "geometry": {"type": "Point", "coordinates": [lon, lat]},
                "properties": {
                    "id": nid,
                    **features.get(nid, {})
                }
            })

    geojson = {
        "type": "FeatureCollection",
        "features": geo_features,
        "metadata": {
            "source": "SWMM simulation via pyswmm",
            "generated": datetime.now().isoformat(),
            "crs": "EPSG:4326"
        }
    }

    with open(output_path, 'w') as f:
        json.dump(geojson, f, indent=2)

    print(f"✅ {kind.capitalize()} GeoJSON saved: {output_path}")


def geojson_export_main(sim_data, network_data, inp_path):
    coords = network_data['coordinates']
    conduits = network_data['conduits']
    vertices = network_data['vertices']
    node_ids = network_data['junctions'] | network_data['storage'] | network_data['outfalls']

    # Prepare max value dictionaries
    conduit_data = {}
    for lid in sim_data['conduit_timeseries']['FLOW']:
        max_flow = max(sim_data['conduit_timeseries']['FLOW'][lid])
        max_depth = max(sim_data['conduit_timeseries']['DEPTH'][lid])
        max_volume = max(sim_data['conduit_timeseries']['VOLUME'][lid])
        conduit_data[lid] = {
            "Max Flow (cfs)": max_flow,
            "Max Depth (ft)": max_depth,
            "Max Volume (ft³)": max_volume
        }

    node_data = {}
    for nid in sim_data['node_data']:
        node_data[nid] = {
            "Max Depth (ft)": max(sim_data['node_data'][nid]['Depth (ft)']),
            "Max Head (ft)": max(sim_data['node_data'][nid]['Head (ft)']),
            "Max Inflow (cfs)": max(sim_data['node_data'][nid]['Total Inflow (cfs)']),
            "Max Lateral Inflow (cfs)": max(sim_data['node_data'][nid]['Lateral Inflow (cfs)']),
            "Max Surcharge (ft)": max(sim_data['node_data'][nid]['Surcharge Depth (ft)'])
        }

    # Load projection info from .ini
    ini_path = Path(inp_path).with_suffix('.ini')
    projection = "ESRI:102698"
    if ini_path.exists():
        with open(ini_path) as f:
            for line in f:
                if 'Projection=' in line:
                    projection = line.split('=')[1].strip()
                    break

    try:
        transformer = Transformer.from_crs(projection, "EPSG:4326", always_xy=True)
    except:
        print("⚠️ Invalid projection. Assuming EPSG:4326.")
        transformer = None

    # Attach data to conduits
    for conduit in conduits:
        conduit['data'] = conduit_data.get(conduit['id'], {})

    # 🔸 Create timestamped folder
    timestamp_folder = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    output_dir = Path("GIS") / timestamp_folder
    os.makedirs(output_dir, exist_ok=True)

    # 🔸 Export GeoJSONs (Optional for users)
    #export_geojson(coords, conduits, vertices, transformer, output_dir / "Conduits.geojson", kind="conduits")
    #export_geojson(coords, node_data, vertices, transformer, output_dir / "Nodes.geojson", kind="nodes")

    # 🔸 Export GeoPackage
    conduits_gdf = gpd.GeoDataFrame(
        [
            {
                "geometry": LineString(
                    [transformer.transform(*coords[conduit['from_node']])] +
                    [transformer.transform(*v) for v in vertices.get(conduit['id'], [])] +
                    [transformer.transform(*coords[conduit['to_node']])]
                ) if transformer else LineString(
                    [coords[conduit['from_node']]] +
                    vertices.get(conduit['id'], []) +
                    [coords[conduit['to_node']]]
                ),
                **{
                    "id": conduit['id'],
                    "from_node": conduit['from_node'],
                    "to_node": conduit['to_node'],
                    **conduit['data']
                }
            }
            for conduit in conduits
            if conduit['from_node'] in coords and conduit['to_node'] in coords
        ],
        crs="EPSG:4326"
    )

    nodes_gdf = gpd.GeoDataFrame(
        [
            {
                "geometry": Point(transformer.transform(*coords[nid])) if transformer else Point(coords[nid]),
                **{"id": nid, **node_data[nid]}
            }
            for nid in node_data if nid in coords
        ],
        crs="EPSG:4326"
    )

    gpkg_path = output_dir / "SWMM_Data.gpkg"
    conduits_gdf.to_file(gpkg_path, layer="Conduits", driver="GPKG")
    nodes_gdf.to_file(gpkg_path, layer="Nodes", driver="GPKG")

    print(f"✅ GeoPackage saved: {gpkg_path}")
    print(f"✅ GeoJSONs saved inside: {output_dir}")

geojson_export_main(sim_data, network_data, inp_path)


✅ GeoPackage saved: GIS/2025-07-25_12-19-29/SWMM_Data.gpkg
✅ GeoJSONs saved inside: GIS/2025-07-25_12-19-29


***🗺️ Interactive Map Visualization***

In [ ]:
import folium
import branca.colormap as cm
from pathlib import Path
from pyproj import Transformer
import numpy as np
from IPython.display import display, clear_output
from ipywidgets import IntSlider, Dropdown, HBox, Output as ipyOutput, VBox
from folium import plugins

def format_datetime(dt):
    rounded_minute = (dt.minute // 5) * 5
    return dt.replace(minute=rounded_minute, second=0).strftime('%Y-%m-%d %H:%M')

def plot_node_and_conduit_attributes(network_data, sim_data, inp_path):
    ini_path = Path(inp_path).with_suffix('.ini')
    projection = 'ESRI:102698'
    if ini_path.exists():
        with open(ini_path) as f:
            for line in f:
                if 'Projection=' in line:
                    projection = line.split('=')[1].strip()
                    break

    try:
        transformer = Transformer.from_crs(projection, "EPSG:4326", always_xy=True)
    except:
        transformer = None

    if transformer:
        coord_map = {nid: transformer.transform(*coords) for nid, coords in network_data['coordinates'].items()}
    else:
        coord_map = network_data['coordinates']
    coord_map = {nid: [lat, lon] for nid, (lon, lat) in coord_map.items()}

    conduit_lines = {}
    for conduit in network_data['conduits']:
        from_coords = coord_map.get(conduit['from_node'])
        to_coords = coord_map.get(conduit['to_node'])
        if not from_coords or not to_coords:
            continue
        line_coords = [from_coords]
        for x, y in network_data['vertices'].get(conduit['id'], []):
            lon, lat = transformer.transform(x, y) if transformer else (x, y)
            line_coords.append([lat, lon])
        line_coords.append(to_coords)
        conduit_lines[conduit['id']] = line_coords

    center_lat = sum(c[0] for c in coord_map.values()) / len(coord_map)
    center_lon = sum(c[1] for c in coord_map.values()) / len(coord_map)

    conduit_attrs = {
        'Flow (cfs)': 'FLOW',
        'Depth (ft)': 'DEPTH',
        'Volume (ft³)': 'VOLUME',
        'Velocity (ft/s)': 'VELOCITY',
        'Capacity': 'CAPACITY'
    }
    node_attrs = {
        'Depth (ft)': 'Depth (ft)',
        'Head (ft)': 'Head (ft)',
        'Total Inflow (cfs)': 'Total Inflow (cfs)',
        'Lateral Inflow (cfs)': 'Lateral Inflow (cfs)',
        'Surcharge Depth (ft)': 'Surcharge Depth (ft)',
        'Flooding (cfs)': 'Flooding (cfs)',
        'Ponded Volume (ft³)': 'Ponded Volume (ft³)'
    }
    units = {
        'FLOW': 'cfs', 'DEPTH': 'ft', 'VOLUME': 'ft³', 'VELOCITY': 'ft/s', 'CAPACITY': '%',
        'Depth (ft)': 'ft', 'Head (ft)': 'ft', 'Total Inflow (cfs)': 'cfs',
        'Lateral Inflow (cfs)': 'cfs', 'Surcharge Depth (ft)': 'ft',
        'Flooding (cfs)': 'cfs', 'Ponded Volume (ft³)': 'ft³'
    }

    base_maps = {
        "CartoDB Dark": folium.TileLayer(tiles='https://{s}.basemaps.cartocdn.com/dark_all/{z}/{x}/{y}.png', attr='CartoDB', name='CartoDB Dark', max_zoom=19, detect_retina=True),
        "ESRI Satellite": folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}', attr='ESRI', name='ESRI Satellite', max_zoom=19, detect_retina=True)
    }

    conduit_dropdown = Dropdown(options=[(k, v) for k, v in conduit_attrs.items() if v in sim_data['conduit_timeseries']], description='Conduit:', style={'description_width': 'initial'})
    node_dropdown = Dropdown(options=[(k, v) for k, v in node_attrs.items() if any(v in d for d in sim_data['node_data'].values())], description='Node:', style={'description_width': 'initial'})
    time_slider = IntSlider(min=0, max=len(sim_data['times']) - 1, step=1, description='Time:', continuous_update=False, readout=False, style={'description_width': 'initial'})
    time_label = ipyOutput()
    output_area = ipyOutput()

    def get_global_min_max(attr, is_conduit=True):
        all_values = []
        if is_conduit:
            for cid in conduit_lines:
                values = sim_data['conduit_timeseries'][attr].get(cid, [None]*len(sim_data['times']))
                all_values.extend([v for v in values if v is not None])
        else:
            for nid in coord_map:
                if nid in sim_data['node_data'] and attr in sim_data['node_data'][nid]:
                    values = sim_data['node_data'][nid][attr]
                    all_values.extend([v for v in values if v is not None])
        if not all_values:
            return 0, 1
        min_val = min(all_values)
        max_val = max(all_values)
        if min_val == max_val:
            min_val -= 0.5
            max_val += 0.5
        return min_val, max_val

    def create_left_legend(title, subtitle, min_val, max_val, colors, unit, position_top):
        def format_number(val):
            rounded = round(val, 1)
            if rounded == int(rounded):
                return str(int(rounded))
            return f"{rounded:.1f}".rstrip('0').rstrip('.') if '.' in f"{rounded:.1f}" else f"{rounded:.0f}"
        values = np.linspace(min_val, max_val, 5)
        labels = [format_number(v) for v in values]
        legend_html = f'''
        <div style="position: fixed; top: {position_top}px; left: 10px; width: 60px; height: 180px;
                    border:1px solid grey; z-index:9997; font-size:10px;
                    background-color:white; padding: 4px; opacity: 0.95;">
            <div style="text-align: center; font-weight: bold; margin-bottom: 2px;">{title}</div>
            <div style="text-align: center; margin-bottom: 4px;">{subtitle}</div>
            <div style="display: flex; flex-direction: row;">
                <div style="display: flex; flex-direction: column; height: 110px; width: 14px; margin-right: 4px;">
                    {''.join(f'<div style="flex: 1; background-color: {colors[i]};"></div>' for i in range(len(colors)))}
                </div>
                <div style="display: flex; flex-direction: column; justify-content: space-between;
                            height: 110px; text-align: left;">
                    {''.join(f'<div style="height: 20%; line-height: 1;">{label}</div>' for label in labels)}
                </div>
            </div>
            <div style="text-align: center; margin-top: 3px;">{unit}</div>
        </div>
        '''
        return legend_html

    def render_map(conduit_attr, node_attr, t_index):
        time = format_datetime(sim_data['times'][t_index])
        cmin, cmax = get_global_min_max(conduit_attr, True)
        nmin, nmax = get_global_min_max(node_attr, False)

        colorscale = ['blue', 'cyan', 'green', 'yellow', 'red']
        node_cmap = cm.LinearColormap(colorscale, vmin=nmin, vmax=nmax)
        conduit_cmap = cm.LinearColormap(colorscale, vmin=cmin, vmax=cmax)

        c_display = next(k for k, v in conduit_attrs.items() if v == conduit_attr)
        n_display = next(k for k, v in node_attrs.items() if v == node_attr)

        m = folium.Map(location=[center_lat, center_lon], zoom_start=16, tiles='openstreetmap', control_scale=True, prefer_canvas=True)
        for name, layer in base_maps.items():
            layer.add_to(m)
        folium.LayerControl(position='topright').add_to(m)

        for cid, coords in conduit_lines.items():
            val = sim_data['conduit_timeseries'][conduit_attr].get(cid, [None]*len(sim_data['times']))[t_index]
            color = conduit_cmap(val) if val is not None else 'gray'
            popup_content = f"<b>Conduit:</b> {cid}<br>"
            for label, key in conduit_attrs.items():
                if key in sim_data['conduit_timeseries']:
                    val_all = sim_data['conduit_timeseries'][key].get(cid, [None]*len(sim_data['times']))[t_index]
                    popup_content += f"<b>{label}:</b> {val_all:.2f} {units.get(key, '')}<br>" if val_all is not None else f"<b>{label}:</b> N/A<br>"
            folium.PolyLine(locations=coords, color=color, weight=3, popup=popup_content).add_to(m)

        for nid, latlon in coord_map.items():
            if nid in sim_data['node_data']:
                val = sim_data['node_data'][nid].get(node_attr, [None]*len(sim_data['times']))[t_index]
                color = node_cmap(val) if val is not None else 'gray'
                popup_content = f"<b>Node:</b> {nid}<br>"
                for label, key in node_attrs.items():
                    if key in sim_data['node_data'][nid]:
                        val_all = sim_data['node_data'][nid][key][t_index]
                        popup_content += f"<b>{label}:</b> {val_all:.2f} {units.get(key, '')}<br>" if val_all is not None else f"<b>{label}:</b> N/A<br>"
                folium.CircleMarker(location=latlon, radius=5, color=color, fill=True, fill_opacity=0.9, popup=popup_content).add_to(m)

        m.get_root().html.add_child(folium.Element(create_left_legend("NODE", n_display.split(' (')[0], nmin, nmax, colorscale, units[node_attr], 120)))
        m.get_root().html.add_child(folium.Element(create_left_legend("LINK", c_display.split(' (')[0], cmin, cmax, colorscale, units[conduit_attr], 310)))

        folium.plugins.Fullscreen(position='topleft', force_separate_button=True).add_to(m)
        folium.plugins.MeasureControl(position='topright', primary_length_unit='feet', secondary_length_unit='miles', active_color='orange', completed_color='red', z_index=9998).add_to(m)
        folium.plugins.MiniMap(position='bottomright', toggle_display=True, zoom_level_fixed=10, minimized=True).add_to(m)

        m.get_root().html.add_child(folium.Element(f'''
        <div style="position: fixed; top: 10px; left: 80px; padding: 6px; border-radius: 4px;
                    background-color: white; border: 1px solid grey;
                    z-index: 9999; font-size: 14px; opacity: 0.95;">
            <b>Time:</b> {time}
        </div>'''))

        with time_label:
            clear_output(wait=True)
            display(time)

        with output_area:
            clear_output(wait=True)
            display(m)

    def on_change(change):
        render_map(conduit_dropdown.value, node_dropdown.value, time_slider.value)

    conduit_dropdown.observe(on_change, names='value')
    node_dropdown.observe(on_change, names='value')
    time_slider.observe(on_change, names='value')

    render_map(conduit_dropdown.value, node_dropdown.value, time_slider.value)

    display(VBox([
        HBox([conduit_dropdown, node_dropdown]),
        HBox([time_slider, time_label]),
        output_area
    ]))

# Example usage:
plot_node_and_conduit_attributes(network_data, sim_data, inp_path="Example.inp")


***🎬 Flow Animation***

In [12]:
def animate_flow_inline(network_data, sim_data, frame_step=100):
    """Create an inline animation of flow through conduits with reduced size"""
    # Prepare coordinates
    coords = {}
    for conduit in network_data['conduits']:
        cid = conduit['id']
        path = []
        path.append(network_data['coordinates'].get(conduit['from_node']))
        path.extend(network_data['vertices'].get(cid, []))
        path.append(network_data['coordinates'].get(conduit['to_node']))
        if None not in path:
            coords[cid] = path

    # Get global min and max flow
    all_vals = [v for sublist in sim_data['conduit_timeseries']['FLOW'].values() 
               for v in sublist if v is not None]
    vmin, vmax = min(all_vals), max(all_vals)
    if vmin == vmax:
        vmin -= 0.1
        vmax += 0.1

    # Use a green-to-red colormap with fewer segments to reduce size
    cmap = colors.LinearSegmentedColormap.from_list("gr", ["green", "yellow", "red"], N=64)
    norm = colors.Normalize(vmin=vmin, vmax=vmax)

    # Adaptive frame sampling
    total_frames = len(sim_data['times'])
    max_frames = 50  # Target maximum frames
    frame_step = max(frame_step, total_frames // max_frames)
    frame_indices = list(range(0, total_frames, frame_step))

    fig, ax = plt.subplots(figsize=(8, 6))  # Reduced figure size
    ax.axis("off")
    title = ax.set_title("Flow Animation", fontsize=12)

    # Efficient node plotting
    nodes_x, nodes_y = zip(*[coord for coord in network_data['coordinates'].values() 
                           if coord is not None])
    ax.plot(nodes_x, nodes_y, 'o', color='blue', markersize=2, alpha=0.7)

    # Initialize conduit lines with optimized properties
    lines = {}
    for cid, path in coords.items():
        xs, ys = zip(*path)
        line, = ax.plot(xs, ys, color='gray', linewidth=1.5, alpha=0.8, solid_capstyle='round')
        lines[cid] = line

    # Colorbar with optimized properties
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax, label='FLOW (cfs)')
    cbar.ax.tick_params(labelsize=8)  # Smaller colorbar labels

    def update(t_index):
        t = frame_indices[t_index]
        title.set_text(f"Flows at time {format_datetime(sim_data['times'][t])}")
        for cid, line in lines.items():
            flow = sim_data['conduit_timeseries']['FLOW'][cid][t]
            color = cmap(norm(flow)) if flow is not None else 'gray'
            line.set_color(color)
        return list(lines.values()) + [title]

    # Create optimized animation
    ani = animation.FuncAnimation(
        fig,
        update,
        frames=len(frame_indices),
        interval=300,  # Slower playback
        blit=True
    )
    
    plt.close(fig)
    
    # Return with increased embed limit
    with plt.rc_context({'animation.embed_limit': 50}):  # 50MB limit
        return HTML(ani.to_jshtml(
            fps=8,  # Reduced frame rate
            embed_frames=True,
            default_mode='loop'
        ))

# Run the animation
animate_flow_inline(network_data, sim_data)

***📈 Hydrograph Widget***

In [6]:

def launch_hydrograph_widget(network_data, sim_data):
    """Interactive widget for viewing hydrographs"""
    entity_selector = Dropdown(options=['Node', 'Conduit'], description='Entity:')
    id_selector = Dropdown(description='ID:')
    attr_selector = Dropdown(description='Attribute:')
    out = Output()

    def update_ids(*args):
        entity = entity_selector.value
        if entity == 'Node':
            id_selector.options = sorted(sim_data['node_data'].keys())
        else:
            id_selector.options = sorted(sim_data['link_data'].keys())

    def update_attributes(*args):
        entity = entity_selector.value
        if entity == 'Node':
            attr_selector.options = [
                'Depth (ft)', 
                'Head (ft)', 
                'Total Inflow (cfs)', 
                'Lateral Inflow (cfs)',
                'Surcharge Depth (ft)'
            ]
        else:
            attr_selector.options = [
                'Flow (cfs)', 
                'Depth (ft)', 
                'Volume (ft³)'
            ]

    def plot_timeseries(change=None):
        with out:
            clear_output(wait=True)
            entity = entity_selector.value
            eid = id_selector.value
            attr = attr_selector.value
            
            # Check if all selections are made
            if None in [entity, eid, attr]:
                print("Please select entity attributes")
                return
                
            plt.figure(figsize=(12, 5))
            plt.title(f"{entity} '{eid}' - {attr}")

            if entity == 'Node':
                plt.plot(sim_data['times'], sim_data['node_data'][eid][attr], label=attr, color='tab:blue')
                if attr == 'Total Inflow (cfs)':
                    plt.plot(sim_data['times'], sim_data['node_data'][eid]['Lateral Inflow (cfs)'], 
                             label='Lateral Inflow (cfs)', color='tab:orange', linestyle='--')
            else:
                plt.plot(sim_data['times'], sim_data['link_data'][eid][attr], label=attr, color='tab:green')

            plt.xlabel("Time")
            plt.ylabel(attr.split(' (')[0] + ' (' + attr.split(' (')[1] if '(' in attr else attr)
            plt.legend()
            plt.grid(True)

            # Add year and time formatting to x-axis
            ax = plt.gca()
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d\n%H:%M'))
            plt.xticks(rotation=45)

            plt.tight_layout()
            plt.show()

    # Connect update and plotting functions to widget events
    entity_selector.observe(update_ids, names='value')
    entity_selector.observe(update_attributes, names='value')
    id_selector.observe(plot_timeseries, names='value')
    attr_selector.observe(plot_timeseries, names='value')

    # Trigger initial update
    update_ids()
    update_attributes()

    # Display the widgets and output plot
    display(VBox([
        HBox([entity_selector, id_selector, attr_selector]),
        out
    ]))

# Usage (uncomment and provide valid data to use):
launch_hydrograph_widget(network_data, sim_data)